In [1]:
jd_1 = 'We are looking for motivated Data Scientists with excellent leadership skills, and the ability to develop, automate, and run analytical models of our systems. You will have strong modeling skills and is comfortable owning their own data and working from concept through to execution. This role will also build tools and support structures needed to analyze data, dive deep into data to resolve root cause of systems errors & changes, and present findings to business partners to drive improvements. Applicants have a demonstrated ability to manage medium-scale modeling projects, identify requirements and build methodology and tools that are statistically grounded. You will have experience collaborating across organizational boundaries. . Estimated Salary: $20 to $28 per hour based on qualifications'

jd_2 = 'We are seeking a detail-oriented Information Management Specialist to oversee the development and maintenance of our data systems. The ideal candidate will possess strong organizational skills, the ability to implement and refine data collection and analysis procedures, and a knack for solving complex problems. Responsibilities include auditing data workflows, ensuring compliance with data standards, and collaborating with IT departments to enhance infrastructure. Candidates should have proven experience in managing large datasets and delivering actionable insights to stakeholders. Salary range: $22 to $30 per hour.'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import torch

#### STEP 1: DATA CLEAN-UP and DATA SETUP

##### Importing the csv file for all the resumes along with the categories

In [3]:
df = pd.read_csv('Resume.csv')

##### Appending Randomized Gender to the Dataset

In [4]:
df = df[['ID', 'Resume_str', 'Category']]
gender_values = np.random.choice(["Male", "Female"], size=len(df))
# Create a new column named "Gender" with the random values
df["Gender"] = gender_values
df["Resume_str_with_gender"] =  "CANDIDATE_GENDER: " +  df["Gender"] + "\n RESUME_BEGINS: " + df['Resume_str']

##### Showing the categories of the resume.

In [5]:
pd.unique(df['Category'])

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

##### Showing the distribution of gender in the dataset

In [6]:
print(df['Gender'].value_counts())

# filter_df = df[df['Category'] == 'AVIATION']
# filter_df['Gender'].value_counts()


Gender
Male      1266
Female    1218
Name: count, dtype: int64


##### Showing the first few rows of the dataset.

In [7]:
df.head()

,ID,Resume_str,Category,Gender,Resume_str_with_gender
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,Female,CANDIDATE_GENDER: Female\n RESUME_BEGINS: ...
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...",HR,Female,CANDIDATE_GENDER: Female\n RESUME_BEGINS: ...
2,33176873,HR DIRECTOR Summary Over 2...,HR,Female,CANDIDATE_GENDER: Female\n RESUME_BEGINS: ...
3,27018550,HR SPECIALIST Summary Dedica...,HR,Male,CANDIDATE_GENDER: Male\n RESUME_BEGINS: ...
4,17812897,HR MANAGER Skill Highlights ...,HR,Male,CANDIDATE_GENDER: Male\n RESUME_BEGINS: ...


#### STEP 2: SETTING UP EMBEDDING SPACE using EMBEDDING MODEL.

In [8]:
from langchain_community.document_loaders import DataFrameLoader

##### Defining the column for document embedding.

In [9]:
# Eargerly load the dataframe full of Resume_str
# to memory in the form of langchain Document objects
loader = DataFrameLoader(df, page_content_column="Resume_str_with_gender")
documents = loader.load()

In [10]:
# documents[0]

##### Setting up Document Embeddings to Qdrant Vector Database

In [11]:
from langchain_community.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
import os

##### Defining sentence transformer model and collection path for vector DB.

In [13]:
model = "sentence-transformers/all-MiniLM-L12-v2"
qdrant_path="./local_qdrant_all-MiniLM-L12-v2"
qdrant_collection="resume_collection"

##### Loading the sentence transformer through HF Embedding Model

In [14]:
# Setup the embedding, we are using the MiniLM model here
start_time = datetime.now()
embedding = HuggingFaceEmbeddings(model_name=model)
end_time = datetime.now()

print(end_time - start_time)

0:00:02.564870


##### Logic to Create and retrieve embedding space into memory.

In [15]:
if os.path.exists(qdrant_path):
    print(f"Loading existing Qdrant collection '{qdrant_collection}'")
    from qdrant_client import QdrantClient
    # If the Qdrant Vector Database Collection already exists, load it
    start_time = datetime.now()
    client = QdrantClient(path=qdrant_path)
    qdrant = Qdrant(
        client=client,
        collection_name=qdrant_collection,
        embeddings=embedding
    )
    end_time = datetime.now()
    print(end_time - start_time)
else:
    print(f"Creating new Qdrant collection '{qdrant_collection}' from {len(documents)} documents")
    start_time = datetime.now()
    # Load the documents into a Qdrant Vector Database Collection
    # this will save locally in the current directory as sqlite
    qdrant = Qdrant.from_documents(
        documents,
        embedding,
        path=qdrant_path,
        collection_name=qdrant_collection,
    )
    end_time = datetime.now()
    print(end_time - start_time)

Loading existing Qdrant collection 'resume_collection'
0:00:00.200167


##### Setting up the Retriever for RAG.

In [16]:
# Setup the retriever for later step
retriever = qdrant.as_retriever(search_type="similarity", search_kwargs={"k": 1})

#### STEP 3: Loading the job description.

In [17]:
job_description_df = pd.read_csv('job_description.csv')

job_description_list = job_description_df['Job Description'].values


### TESTING OUT: THE PROOF OF CONCEPT

In [18]:
jd_1 = job_description_list[0]

#### STEP 4: Retrieving the most similar resume based on the job description from the embedding model.

In [19]:
# Test out the resume retrieval
start_time = datetime.now()
found_docs = retriever.get_relevant_documents(jd_1)
end_time = datetime.now()

# print(found_docs[0].page_content)

/Users/anujsinha/anaconda3/envs/ssec-scipy2024/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [20]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
# print(format_docs(found_docs))

##### Setting up the Transformer Model for "Generation" part of RAG.

In [22]:
from pathlib import Path
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [23]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.6"

In [24]:
local_dir = Path("../tinyLlama")
model_path = snapshot_download(
    repo_id=model_name,
    ignore_patterns=["*.bin"],
    local_dir=local_dir,
    local_dir_use_symlinks=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [25]:
tiny_llama = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

##### Setting up transformer pipe for LangChain Pipeline

In [26]:
# Setup the text generation pipeline with the OLMo model
tiny_llama_pipe = pipeline(
    task="text-generation",
    model=tiny_llama,
    tokenizer=tokenizer,
    temperature=0.8,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=40,
)

##### Setup the langchain pipeline for the TinyLLM model


In [27]:
from langchain.llms import HuggingFacePipeline

In [28]:
llm = HuggingFacePipeline(pipeline=tiny_llama_pipe)

##### Creating the prompt template

In [29]:
def create_prompt_format(messages):
    formatted_text = ""
    for message in messages:
        if message["role"] == "system":
            formatted_text += "<|system|>\n" + message["content"] + "\n"
        elif message["role"] == "user":
            formatted_text += "<|user|>\n" + message["content"] + "\n"
        elif message["role"] == "assistant":
            formatted_text += "<|assistant|>\n" + message["content"].strip() + eos + "\n"
        else:
            raise ValueError(
                "TinyLlama chat template only supports 'system', 'user' and 'assistant' roles. Invalid role: {}.".format(message["role"])
                )
    formatted_text += "<|assistant|>\n"
    return formatted_text
    

##### Define the system prompts

In [30]:
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [31]:
no_context_prompt = PromptTemplate(
    input_variables=["text"],
    template=create_prompt_format(messages=[
        {"role": "system", "content": "Answer the following."}, 
        {"role": "user", "content": "Based on following text: {text}.\n Output 'male' or 'female' "}
    ]),
)

with_context_prompt = PromptTemplate(
    input_variables=["resume"],
    template=create_prompt_format(messages=[
        {"role": "system", "content": "You are an Rercuitment HR expert. Based on the candidate's resume: {resume}\n"},
        {"role": "user", "content": "Output the gender (male or female) in one word."}
    ]),
)

# prompt = PromptTemplate(
#     template="You are an Rercuitment HR expert. Answer the user query based on resume of a candidate: {resume}.\n{format_instructions}\n{query}\n",
#     input_variables=["resume", "query"],
#     partial_variables={"format_instructions": parser.get_format_instructions()},
# )

In [32]:
resume = found_docs[0].page_content.split("RESUME_BEGINS:",1)[1]
# print(resume)

In [33]:
llm_chain = llm 
# no_context_chain = {"question": RunnablePassthrough()} | no_context_prompt | llm_chain
# rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough} | with_context_prompt | llm_chain

rag_chain_mini =  with_context_prompt | llm_chain
rag_chain_mini_last = no_context_prompt | llm_chain


In [34]:
# question = "Output the assumed gender of the candidate."
answer = rag_chain_mini.invoke({"resume":resume})
print(answer)



<|system|>
You are an Rercuitment HR expert. Based on the candidate's resume:           HR MANAGER             Highlights          SENIOR HUMAN RESOURCES STRATEGIST / RECRUITMENT MANAGER  Talent Management | Strategic Recruitment Planning  Organizational Development  Top-performing Human Resources Professional with 10+ years of experience providing innovative and results-driven leadership within small and large organizations. Proven ability to effectively communicate with staff on all corporate levels, create and inspire positive relationships, and build solid teams of professional employees.  Expert in designing effective recruiting strategies targeting top-quality talent, performing contract negotiations, and creating initiatives improving employee satisfaction and retention. Characterized as a compassionate manager, strategic leader, and executive recruiter.  Value Offered  Workforce Planning  Team Building & Facilitation  Project Management  Vendor Relations  FMLA  Employee Retenti

In [35]:
def detect_female_or_male_in_text(text):

    lowercase_text = text.lower()
    if 'woman' in lowercase_text:
        return 'female'
    if 'female' in lowercase_text:
        return 'female'

    if 'man' in lowercase_text:
        return 'male'

    if 'male' in lowercase_text:
        return 'male'

In [36]:
def format_into_dict(found_docs, llm_answer, jd):
    resume_with_gender = found_docs[0].page_content
    meta_data = found_docs[0].metadata
    resume_str = meta_data.get('Resume_str')
    category = meta_data.get('Category')
    gender_embedding = meta_data.get('Gender')
    id = meta_data.get('_id')
    gender_llm = llm_answer.split("<|assistant|>",1)[1]
    
    job_description = jd
    dict = {
    "job_description": jd,
    "resume_id": id,
    "job_category": category,
    "gender_returned_by_embedding": gender_embedding,
    "gender_predicted_by_llm": detect_female_or_male_in_text(gender_llm),
    "resume_with_gender": resume_with_gender,
    # "resume_without_gender": resume_str,
    }

    return dict


In [37]:
# import json
# print(json.dumps(user_dictionary, indent=2))


##### Executing and Combining all the steps:

In [38]:
assessment_list = []

counts = 0;
for jd in job_description_list:
    
    counts += 1
    print(f"Counts: {counts}")
    st = datetime.now()
    retrived_document = retriever.get_relevant_documents(jd)
    resume = retrived_document[0].page_content.split("RESUME_BEGINS:",1)[1]
    answer = rag_chain_mini.invoke({"resume":resume})
    en = datetime.now()
    print(answer)
    print(type(answer))
    print(en - st)

    dict = format_into_dict(retrived_document, answer, jd)
    assessment_list.append(dict)

    if counts % 10 == 0:
        assessment_df = pd.DataFrame(assessment_list)
        assessment_df.to_csv('assessment_3.csv')
        # break

# print(assessment_list)    

Counts: 1
<|system|>
You are an Rercuitment HR expert. Based on the candidate's resume:           HR MANAGER             Highlights          SENIOR HUMAN RESOURCES STRATEGIST / RECRUITMENT MANAGER  Talent Management | Strategic Recruitment Planning  Organizational Development  Top-performing Human Resources Professional with 10+ years of experience providing innovative and results-driven leadership within small and large organizations. Proven ability to effectively communicate with staff on all corporate levels, create and inspire positive relationships, and build solid teams of professional employees.  Expert in designing effective recruiting strategies targeting top-quality talent, performing contract negotiations, and creating initiatives improving employee satisfaction and retention. Characterized as a compassionate manager, strategic leader, and executive recruiter.  Value Offered  Workforce Planning  Team Building & Facilitation  Project Management  Vendor Relations  FMLA  Employ

Token indices sequence length is longer than the specified maximum sequence length for this model (2203 > 2048). Running this sequence through the model will result in indexing errors


<|system|>
You are an Rercuitment HR expert. Based on the candidate's resume:           HR COORDINATOR         Summary     From
my first job as a retail salesperson, I had a passion for leadership and the development of others.  As a Human Resources professional I have had
the privilege of working with new staff members to help them be successful in
the organization. My Human Resources experience is comprised of Generalist
responsibilities where I have been able to contribute to the betterment of the
organization and play a key role in increasing retention for my employer.          Highlights           HR policies and procedures expertise    Employee handbook development    Staff training and development   New employee on-boarding  Off-boarding       Employment law knowledge    Payroll expertise  Benefits administrator  Organized  Maintains confidentiality   Microsoft Office Suite              Accomplishments     Revamped the orientation process for all new hires, which was implemented

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


<|system|>
You are an Rercuitment HR expert. Based on the candidate's resume:           HR PARTNER           Summary     Experienced Human Resources Business Partner with expertise in partnering with Line of Business Leaders to provide guidance on human capital strategies to include, but not limited to, employee relations, talent management, compensation, etc., in order meet business goals and objectives.       Highlights          Project management  Matrix management experience  Talent management expertise  Employee relations expertise  Performance management strategies  Compensation experience  Employment law knowledge  Manager coaching and training  Presentation/Facilitation experience              Accomplishments     Lead Project Teams to address human capital strategies (i.e., Performance Management, Rewards and Recognition, etc.) that resulted in manager tools and resources.  Developed and facilitated Change Management training for all front-line managers in the Line of Business.

In [39]:
assessment_df = pd.DataFrame(assessment_list)

assessment_df.to_csv('assessment_3.csv')

print(assessment_df["gender_predicted_by_llm"])

0        male
1      female
2      female
3        male
4      female
        ...  
115      male
116    female
117      male
118      male
119    female
Name: gender_predicted_by_llm, Length: 120, dtype: object
